<a href="https://colab.research.google.com/github/ivansst773/Aprendizaje_de_Maquina/blob/main/Proyecto/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=89c62078ad8fb3c2c1249d8293567644f2c48a8ec615a5bed4755f522959f6c1
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


conectar tu Google Drive:


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


descomprimir todos los archivos dentro del ZIP

In [5]:
from google.colab import drive
import zipfile, os, glob

# Montar Drive
drive.mount('/content/drive')

ruta_zip = "/content/drive/MyDrive/Semestre2025_1/Teoria de aprendizaje de maquina/a-comprehensive-dataset-of-pattern-electroretinograms-for-ocular-electrophysiology-research-the-perg-ioba-dataset-1.0.0.zip"
ruta_destino = "/content/perg-ioba-dataset/"

# Extraer ZIP solo si no se ha hecho
if not os.path.exists(ruta_destino) or len(os.listdir(ruta_destino)) == 0:
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        zip_ref.extractall(ruta_destino)

# Detectar subcarpeta con los datos
subcarpetas = [os.path.join(ruta_destino, f) for f in os.listdir(ruta_destino) if os.path.isdir(os.path.join(ruta_destino, f))]
ruta_base = subcarpetas[0]  # Asume que hay una sola subcarpeta principal

# Buscar CSVs
csv_files = glob.glob(os.path.join(ruta_base, "*.csv"))
print(f"Se encontraron {len(csv_files)} archivos CSV en {ruta_base}")


Archivos extraídos en: /content/perg-ioba-dataset/
['a-comprehensive-dataset-of-pattern-electroretinograms-for-ocular-electrophysiology-research-the-perg-ioba-dataset-1.0.0']


procesa archivos CSV de mediciones PERG y les asigna un identificador (id_record) basado en el nombre del archivo.

📌 Resultado:

✅ Cada archivo CSV se convierte en un DataFrame con su información de medición y su identificador único (id_record).

✅ Permite unir todos los datos en df_mediciones para su análisis posterior.


In [9]:
import glob

# ✅ Ruta corregida
ruta_base = "/content/perg-ioba-dataset/a-comprehensive-dataset-of-pattern-electroretinograms-for-ocular-electrophysiology-research-the-perg-ioba-dataset-1.0.0/"

# Buscar archivos CSV ahora en la ruta correcta
csv_files = glob.glob(ruta_base + "*.csv")
print(f"Se encontraron {len(csv_files)} archivos CSV")


Se encontraron 0 archivos CSV


In [8]:
dataframes = []
for file in csv_files:
    if "participants_info" in file:  # Excluir el archivo de información de participantes
        continue

    id_record = os.path.basename(file).split(".")[0]  # Extraer ID desde el nombre del archivo
    df = pd.read_csv(file)
    df["id_record"] = int(id_record)  # Asignar ID al dataset
    dataframes.append(df)

Este código fusiona todos los DataFrames en un único dataset (df_mediciones) y verifica la asignación de id_record y valores faltantes (NaN).


In [ ]:
# Unir todos los DataFrames en un solo dataset
df_mediciones = pd.concat(dataframes, ignore_index=True)

# Revisar si id_record está correctamente asignado
print("Primeras filas después de asignar id_record:")
print(df_mediciones.head())

print("\nConteo de valores NaN en df_mediciones:")
print(df_mediciones.isna().sum())

📍 Próximo paso: Fusionar df_mediciones con df_info utilizando id_record
Ejecuta esto para unir los datos de medición con la información clínica:


In [ ]:
df_completo = df_mediciones.merge(df_info, on="id_record", how="left")

print("Datos combinados correctamente:")
print(df_completo.head())

print("\nConteo de valores NaN en df_completo:")
print(df_completo.isna().sum())

cuenta cuántos pacientes tienen todas sus mediciones PERG como NaN, lo que significa que no tienen datos de respuesta electrofisiológica registrados.


In [ ]:
print("Número de pacientes con todas las mediciones PERG como NaN:",
      df_completo[df_completo[["RE_1", "LE_1", "RE_2", "LE_2", "RE_3", "LE_3", "RE_4", "LE_4", "RE_5", "LE_5"]].isna().all(axis=1)].shape[0])

Revisar diagnosis2 y diagnosis3
Primero, ejecuta esto para ver cuáles son los valores más comunes en estas columnas:


In [ ]:
print("Valores únicos en diagnosis2:")
print(df_completo["diagnosis2"].value_counts())

print("\nValores únicos en diagnosis3:")
print(df_completo["diagnosis3"].value_counts())

Si prefieres rellenar con "Desconocido"


Este código limpia los valores faltantes (NaN) en las columnas diagnosis2 y diagnosis3, reemplazándolos por "Desconocido".


In [ ]:
df_completo = df_completo.copy()  # Asegurar que estamos modificando el DataFrame original

df_completo["diagnosis2"] = df_completo["diagnosis2"].fillna("Desconocido")
df_completo["diagnosis3"] = df_completo["diagnosis3"].fillna("Desconocido")

Este código cuenta cuántos registros tienen valores NaN en las columnas unilateral, rep_record y comments.


In [ ]:
print("Número de registros con NaN en unilateral:", df_completo["unilateral"].isna().sum())
print("Número de registros con NaN en rep_record:", df_completo["rep_record"].isna().sum())
print("Número de registros con NaN en comments:", df_completo["comments"].isna().sum())

Este código elimina valores NaN en las columnas unilateral, rep_record y comments, reemplazándolos con "Desconocido".


In [ ]:
df_completo = df_completo.copy()  # Asegurar que estamos modificando el DataFrame original

df_completo["unilateral"] = df_completo["unilateral"].fillna("Desconocido")
df_completo["rep_record"] = df_completo["rep_record"].fillna("Desconocido")
df_completo["comments"] = df_completo["comments"].fillna("Desconocido")

Este código proporciona un resumen detallado del dataset df_completo y estadísticas de las mediciones PERG.


In [ ]:
print("Resumen del dataset:")
print(df_completo.info())

print("\nEstadísticas de las mediciones:")
print(df_completo.describe())

🚀 Vamos a construir un modelo de clasificación para predecir diagnosis1 con base en las mediciones PERG y los datos clínicos


🔹 📍 ¿Qué es un modelo de clasificación en Machine Learning?

✅ Un modelo de clasificación es un algoritmo que aprende a asignar etiquetas a datos nuevos, basándose en patrones encontrados en datos previos.

✅ En nuestro caso, el modelo analizará las mediciones PERG (RE_1, LE_1, RE_2, etc.) junto con variables clínicas (age_years, va_re_logMar, etc.) para predecir el diagnóstico (diagnosis1).

✅ La meta es que, dado un nuevo paciente con sus mediciones, el modelo pueda decir qué diagnóstico es más probable.


# ✅ La meta es que, dado un nuevo paciente con sus mediciones, el modelo pueda decir qué diagnóstico es más probable.

🔹 📍 Tipos de modelos de clasificación
Existen muchos algoritmos de clasificación, pero aquí hay algunos populares:

📌 Random Forest → Usa múltiples árboles de decisión para mejorar la precisión.

📌 Red Neuronal Artificial (MLP) → Aprende patrones en datos complejos, similar a cómo lo hace el cerebro humano.

📌 Regresión Logística → Modelo estadístico simple pero efectivo para problemas binarios o multinivel.

📌 K-Nearest Neighbors (KNN) → Predice en función de los datos más cercanos en el espacio de características.

📌 Support Vector Machines (SVM) → Encuentra la mejor separación entre clases con hiperplanos en dimensiones múltiples.


🔹 📍 ¿Cómo funciona el aprendizaje en un modelo de clasificación?

🔹 Fase de entrenamiento:

✅ Se le dan datos etiquetados al modelo (X_train con mediciones + y_train con diagnosis1).

✅ El modelo encuentra patrones en los datos y ajusta sus parámetros internos.

🔹 Fase de predicción:

✅ Con nuevos datos (X_test), el modelo usa lo aprendido para asignar un diagnóstico (y_pred).

✅ Se compara con los valores reales (y_test) y se mide la precisión del modelo.

🔹 Evaluación del modelo:

✅ Se usan métricas como accuracy, precision, recall y F1-score para medir su desempeño.

✅ También podemos usar una matriz de confusión para ver cuántas predicciones fueron correctas o incorrectas.


🔹 📍 ¿Cuál usamos en este proyecto?

📌 Random Forest es una gran opción inicial porque es preciso, fácil de interpretar y funciona bien con datos tabulares.


🔹 📍 Código para preparar los datos


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Selección de características y etiquetas
features = ["RE_1", "LE_1", "RE_2", "LE_2", "age_years", "va_re_logMar", "va_le_logMar"]
X = df_completo[features]  # Variables predictoras
y = df_completo["diagnosis1"]  # Etiqueta de clasificación

# Codificación de etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalización de las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Datos listos para el entrenamiento del modelo de clasificación!")
print("Shape de X_train:", X_train_scaled.shape)
print("Shape de X_test:", X_test_scaled.shape)


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

#1. 🤖 Enseñar al modelo (Random Forest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_rf = RandomForestClassifier(random_state=42)
modelo_rf.fit(X_train_scaled, y_train)


#2. 🧠 Ver cómo responde el robot

In [ ]:
y_pred = modelo_rf.predict(X_test_scaled)


#3. 📊 Evaluar qué tan bien lo hizo

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

matriz = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10,6))
sns.heatmap(matriz, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.savefig('matriz_confusion.pdf')  # Para llevar a Overleaf
plt.show()

#4. 🌟 Ver qué variables usó más el robot para decidir

In [ ]:
import pandas as pd

importancias = modelo_rf.feature_importances_
features = ["RE_1", "LE_1", "RE_2", "LE_2", "age_years", "va_re_logMar", "va_le_logMar"]
df_importancias = pd.DataFrame({'Feature': features, 'Importancia': importancias})
print(df_importancias.sort_values(by='Importancia', ascending=False))
